### Modeling information source transitions

We assume that each individual $i$ stochastically switch between two different information sources A and B. For simplicity, we assume a random switching between the two sources to be described by the so-called dichotomous Markov process, also known as telegraph or Kac process, widely used to model different biological phenomena \[see e.g. Potoyan & Wolynes, Dichotomtous noise models of gene switches, J Chem Phys 143 (2015)\], also in the context of collective movement states \[Bazazi et al, Nutritional state and collective motion: from individuals to mass migration, Proc Roy Soc B 278 (2010)\].

The probabilities per unit time for a single individual to switch from source A to source B and vice versa are constant and determined by constant probabiluity transition rates $\lambda_{A\to B}$  and $\lambda_{B\to A}$. The probability to find a single individual "listening" to one of the sources over time, $p_{A}$ and $p_{B}$ obey the following master eqations: 

$$ \frac{dp_{A}}{dt} = -\lambda_{A\to B} p_{A} + \lambda_{B \to A} p_{B}\ ,$$
$$ \frac{dp_{B}}{dt} = +\lambda_{A\to B} p_{A} - \lambda_{B \to A} p_{B}\ .$$

The first term on the right hand side describes the transition from A to B. It is given by the product of the transition rate $\lambda_{A\to B}$ with the probability to observe the individual in state A. The second term accounts in an analogous way for the reverse transition B to A. The individual has to attend to one of the sources
, thus the additional constrain must hold at all times: $p_A(t)+p_B(t)=1$. 

For such a dichotomous Markov process the time duration an individual stays in one of the states are exponentially distributed, and the average time spent in one of the states is given by the inverse of the corresponding transiton rate out of this state:

$$ \tau_{A} = \frac{1}{\lambda_{A\to B}},\quad  \tau_{B} = \frac{1}{\lambda_{B\to A}}\ .$$

The probability to find an individual in one of the states over time, or the fraction of individuals in a group in one of the state at a certain point in time, is given by the stationary probabilities $p^*_{A}$ and $p^*_{B}$, that can be calculated from $dp_{A}/dt=dp_{B}/dt=0$:

$$ p^*_{A}=1-p^*_{B}=\frac{\lambda_{B \to A}}{\lambda_{A\to B} + \lambda_{B \to A}}=\frac{1}{1+\frac{\tau_{B}}{\tau_{A}}}\ .$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

def DichotomousMarkovProcess(time_steps,rate01=1,rate10=1,dt=0.02):
    ''' Simple function to generate a single realization of a dichotomous Markov switchting process'''
    state=np.zeros(len(time_steps));
    for s in time_steps[1:]:
        rnd=np.random.uniform()
        if(state[s-1]==0):
            state[s]=0;
            if(rnd<rate01*dt):
                state[s]=1;
        else:
            state[s]=1;
            if(rnd<rate10*dt):
                state[s]=0;
        
    return state

In [ ]:
simtime=10
dt=0.02
rate01=1
rate10=1
time_steps=np.arange(0,int(simtime/dt))
state=DichotomousMarkovProcess(time_steps,rate01,rate10,dt)

In [ ]:
plt.figure(figsize=(4,2))
plt.plot(time_steps*dt,state,'-')
plt.xlim([0,simtime])
plt.xlabel('time')
plt.ylabel('state')
plt.yticks([0,1])
#plt.savefig('example_DMP.png',dpi=200)

In [ ]:
@jit(nopython=True)
def SingleStep(past_state,rate01,rate10,dt):
    N=len(past_state)
    random_number=np.random.random(N)
    new_state=np.zeros(N)
    
    for n in range(N):
        # depending on the state set the right transition rate
        if(past_state[n]==-1):
            rate=rate01
        else:
            rate=rate10
            
        new_state[n]=past_state[n]    
        # check whether transition occurs and if so swap sign of the state    
        if(random_number[n]<rate*dt):
                new_state[n]*=-1
       
    return new_state

def collectiveDMP(initial_state,rate01,rate10,time_steps,dt=0.1):
    ''' Simple Run Generating N dimensional / N agents DMP '''
    N=len(initial_state)
    state=np.zeros((int(time_steps),N))
    print(np.shape(state))
    state[0,:]=initial_state
    for s in range(1,int(time_steps)):
        state[s,:]=SingleStep(state[s-1],rate01,rate10,dt)
        
    return state
        

In [ ]:
N=10
rate01=2.0
rate10=1.0
dt=0.1
init_state=np.ones(N)
random_array=np.random.random(N)
p_0=rate10/(rate01+rate10)
init_state[random_array<p_0]=-1
print(p_0,np.sum(init_state<0)/N)

In [ ]:
time_steps=200
states_vs_time=collectiveDMP(init_state,rate01,rate10,time_steps,dt)

In [ ]:
plt.pcolor(states_vs_time.T,cmap='bwr')
plt.xlabel('time')
plt.ylabel('agent idx')
